In [16]:
#from IPython import clear_output
from IPython.display import clear_output
!bash -c "source ~/catkin_ws/devel/setup.bash && env"
clear_output()

In [17]:

import os
import rospy
import sys
import time
import numpy as np
import cv2
import math
import scipy
import rosbag


In [ ]:
def read_bags(root_dir='User_data_with_video', limit = 300):
    """
    Extracts data from all ROS bag files under subfolders user_<number> within the given root directory.
    
    The returned dictionary has the form:
        data[user_number][topic_name] = list of message objects
    
    :param root_dir: The directory containing subfolders named user_<number> with .bag files.
    :return: A nested dictionary of the extracted data.
    """
    
    data = {}
    id_index = []
    cnt = 0
    # List all items in root_dir
    for folder_name in os.listdir(root_dir):
        # We are only interested in folders that start with "user_"
        if folder_name.startswith("user_"):
            cnt += 1
            # Extract the user number from folder name (e.g., user_100 -> "100")
            user_number = folder_name.split("_", 1)[1]
            id_index.append(user_number)
            user_dir = os.path.join(root_dir, folder_name)
            
            # Make sure it's actually a directory
            if not os.path.isdir(user_dir):
                continue
            
            # Initialize a dictionary for this user
            data[user_number] = {}
            
            # Find all .bag files inside this user directory
            bag_files = [
                f for f in os.listdir(user_dir)
                if f.endswith(".bag") and os.path.isfile(os.path.join(user_dir, f))
            ]
            
            for bag_file in bag_files:
                bag_path = os.path.join(user_dir, bag_file)
                
                print(f"Processing bag: {bag_path}")
                
                # Open the bag
                with rosbag.Bag(bag_path, 'r') as bag:
                    # Read all messages (topic, message, timestamp)
                    for topic, msg, t in bag.read_messages():
                        # If topic is not yet in data dict, initialize it
                        # if the topic contains camera_ then skip it
                        if topic not in data[user_number] and topic.find('camera') == -1:
                            data[user_number][topic] = []
                        
                        # Append the message to the list for this topic
                        if topic.find('camera') == -1:
                            data[user_number][topic].append(msg)
            if cnt > limit:
                break
    
    return data, id_index




data, id_index = read_bags('User_data_with_video')
print("Data extraction complete.")
print(f"Extracted data for {len(data)} users.")
print(f"User IDs: {id_index}")
print(f"Data keys: {data[id_index[0]].keys()}")





Processing bag: User_data_with_video/user_185/trial_data.bag
Processing bag: User_data_with_video/user_249/trial_data.bag
Processing bag: User_data_with_video/user_310/trial_data.bag
Processing bag: User_data_with_video/user_234/trial_data.bag
Processing bag: User_data_with_video/user_211/trial_data.bag
Processing bag: User_data_with_video/user_266/trial_data.bag
Processing bag: User_data_with_video/user_209/trial_data.bag
Processing bag: User_data_with_video/user_292/trial_data.bag
Processing bag: User_data_with_video/user_312/trial_data.bag
Processing bag: User_data_with_video/user_260/trial_data.bag
Processing bag: User_data_with_video/user_246/trial_data.bag
Data extraction complete.
Extracted data for 11 users.
User IDs: ['185', '249', '310', '234', '211', '266', '209', '292', '312', '260', '246']
Data keys: dict_keys(['/my_gen3_lite/base_feedback/joint_state', '/my_gen3_lite/base_feedback', '/my_gen3_lite/in/cartesian_velocity', '/my_gen3_lite/joint_states', '/tf', '/joy'])


In [33]:

def write_bags(data, id_index, output_dir='User_data_output'):
    """
    Writes the data from the nested dictionary to ROS bag files, creating one bag per user.
    
    :param data: The nested dictionary returned by read_bags(), 
                 structured as data[user_number][topic_name] = list of message objects.
    :param id_index: The list of user IDs.
    :param output_dir: The directory to store the newly created user_<id> folders and bag files.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Iterate through each user in our data
    for user_number in id_index:
        
        # Create a subdirectory for this user
        user_folder_name = f"user_{user_number}"
        user_dir = os.path.join(output_dir, user_folder_name)
        if not os.path.exists(user_dir):
            os.makedirs(user_dir)
        
        # We'll create a single bag file per user. Adjust naming as you wish.
        bag_file_name = f"user_{user_number}.bag"
        bag_path = os.path.join(user_dir, bag_file_name)
        
        print(f"Writing bag for user_{user_number} at: {bag_path}")
        
        # Open the bag for writing
        with rosbag.Bag(bag_path, 'w') as bag:
            # For each topic, write all messages into the bag
            for topic in data[user_number].keys():
                msg_list = data[user_number][topic]
                # If the original code doesn't store timestamps, create dummy ones.
                for idx, msg in enumerate(msg_list):
                    # Create a fake timestamp
                    # e.g., incremental or based on index
                    stamp = rospy.Time.from_sec(idx * 0.01)
                    bag.write(topic, msg, stamp)
                    
        print(f"Finished writing bag for user_{user_number}.\n")

write_bags(data, id_index, 'video_free_bags')

Writing bag for user_185 at: video_free_bags/user_185/user_185.bag
Finished writing bag for user_185.

Writing bag for user_249 at: video_free_bags/user_249/user_249.bag
Finished writing bag for user_249.

Writing bag for user_310 at: video_free_bags/user_310/user_310.bag
Finished writing bag for user_310.

Writing bag for user_234 at: video_free_bags/user_234/user_234.bag
Finished writing bag for user_234.

Writing bag for user_211 at: video_free_bags/user_211/user_211.bag
Finished writing bag for user_211.

Writing bag for user_266 at: video_free_bags/user_266/user_266.bag
Finished writing bag for user_266.

Writing bag for user_209 at: video_free_bags/user_209/user_209.bag
Finished writing bag for user_209.

Writing bag for user_292 at: video_free_bags/user_292/user_292.bag
Finished writing bag for user_292.

Writing bag for user_312 at: video_free_bags/user_312/user_312.bag
Finished writing bag for user_312.

Writing bag for user_260 at: video_free_bags/user_260/user_260.bag
Finish